In [ ]:
import pandas as pd

customers = pd.DataFrame({
    "CustomerId": [1, 2, 3, 4, 5],
    "FirstName": ["Alice", "Bob", "Carol", "Dave", "Eve"],
    "LastName": ["Smith", "Jones", "Lee", "Brown", "Davis"]
})

invoices = pd.DataFrame({
    "InvoiceId": [101, 102, 103, 104, 105, 106, 107, 108],
    "CustomerId": [1, 2, 1, 3, 4, 2, 5, 5],
    "Total": [20.0, 35.0, 50.0, 40.0, 60.0, 45.0, 30.0, 25.0]
})

invoice_items = pd.DataFrame({
    "InvoiceId": [101, 102, 103, 104, 105, 106, 107, 108],
    "TrackId": [1, 2, 3, 4, 5, 6, 1, 5]
})

tracks = pd.DataFrame({
    "TrackId": [1, 2, 3, 4, 5, 6],
    "AlbumId": [201, 202, 201, 203, 204, 202]
})

albums = pd.DataFrame({
    "AlbumId": [201, 202, 203, 204],
    "Title": ["Rock Album", "Jazz Album", "Pop Album", "HipHop Album"]
})

invoice_with_customers = pd.merge(invoices, customers, on="CustomerId")
customer_totals = invoice_with_customers.groupby(["CustomerId", "FirstName", "LastName"])["Total"].sum().reset_index()
top5_customers = customer_totals.sort_values("Total", ascending=False).head(5)
top5_customers["CustomerName"] = top5_customers["FirstName"] + " " + top5_customers["LastName"]
top5_result = top5_customers[["CustomerId", "CustomerName", "Total"]]

purchases = pd.merge(invoice_items, invoices[["InvoiceId", "CustomerId"]], on="InvoiceId")
purchases = pd.merge(purchases, tracks[["TrackId", "AlbumId"]], on="TrackId")

def buys_full_album(group):
    album_track_counts = tracks.groupby("AlbumId")["TrackId"].count()
    customer_album_counts = group.groupby("AlbumId")["TrackId"].nunique()
    return (customer_album_counts == album_track_counts[customer_album_counts.index]).any()

album_pref_summary = purchases.groupby("CustomerId").apply(buys_full_album)
album_pref_df = album_pref_summary.reset_index(name="BuysFullAlbum")
album_pref_df["Preference"] = album_pref_df["BuysFullAlbum"].map({True: "Full Albums", False: "Individual Tracks"})
pref_percent = album_pref_df["Preference"].value_counts(normalize=True) * 100
pref_result = pref_percent.reset_index()
pref_result.columns = ["Preference", "Percentage"]

print("Top 5 Customers:")
print(top5_result)

print("\nCustomer Preferences (%):")
print(pref_result)
